In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from collections import Counter

# Natural language Toolkit
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.stem import SnowballStemmer                                   # Porter's II Stemmer
from nltk import word_tokenize                                          # Document tokenizer
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.metrics import f1_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, Embedding, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
directory_dominik = 'Genre Classification Dataset'
directory = directory_dominik

filename ="train_data.txt"

df_train = pd.read_csv(os.path.join(directory, filename), delimiter=":::", encoding='utf-8', header=None)

df_train.drop(columns=df_train.columns[0], axis=1, inplace=True)
df_train.columns =['title', 'genre', 'description']
df_train['genre']=df_train['genre'].apply(lambda x: x.strip())

df_train.head()

C:\Users\nikit\AppData\Local\Temp\ipykernel_3364\2348343198.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_train = pd.read_csv(os.path.join(directory, filename), delimiter=":::", encoding='utf-8', header=None)


,title,genre,description
0,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


Use existing Bert, because no additional tuning was performed, dont even need testing dataset 

In [14]:
from transformers import pipeline
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from sklearn.metrics import classification_report
df = pd.DataFrame(df_train)

# Extract unique genres
genre_list = df['genre'].unique().tolist()

# Zero-shot classification pipeline
classifier = pipeline('zero-shot-classification')

# Function to predict genre
def predict_genre(text):
    result = classifier(text, genre_list)
    return result['labels'][0]

# Predict genres for all descriptions in the dataset
df['predicted_genre'] = df['description'].apply(predict_genre)

# Calculate and print classification report
print(classification_report(df['genre'], df['predicted_genre'], labels=genre_list))

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


KeyboardInterrupt: 

At this stage we teach our own BERT takes looooong will fix later

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df.genre.unique()))

# Label encoding
label_encoder = LabelEncoder()
df['genre_encoded'] = label_encoder.fit_transform(df.genre)

# Split data into training and testing
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Tokenize dataset
train_encodings = tokenizer(train_df.description.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_df.description.tolist(), truncation=True, padding=True)

# Dataset class
class GenreDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset objects
train_dataset = GenreDataset(train_encodings, train_df.genre_encoded.tolist())
test_dataset = GenreDataset(test_encodings, test_df.genre_encoded.tolist())

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_dir='./logs',
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained('./fine_tuned_bert')

# Load the fine-tuned model for prediction
model = BertForSequenceClassification.from_pretrained('./fine_tuned_bert')

# Function to predict genre
def predict_genre(text):
    inputs = tokenizer(text, return_tensors='pt')
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).tolist()
    return label_encoder.inverse_transform(predictions)

# Predict the genre for a new description
new_description = "A thrilling story about a detective trying to solve a murder mystery."
predicted_genre = predict_genre(new_description)
print("Predicted genre:", predicted_genre[0])


C:\Users\nikit\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nikit\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/54215 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Function to get predictions
def get_predictions(dataset):
    predictions = []
    true_labels = []

    for item in dataset:
        inputs = {key: torch.tensor(val).unsqueeze(0) for key, val in item.items() if key != 'labels'}
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1).tolist()
        predictions.extend(prediction)
        true_labels.extend(item['labels'].tolist())

    return label_encoder.inverse_transform(predictions), label_encoder.inverse_transform(true_labels)

# Get predictions for train and test datasets
train_preds, train_true = get_predictions(train_dataset)
test_preds, test_true = get_predictions(test_dataset)

# Calculate and print classification report
print("Training classification report:")
print(classification_report(train_true, train_preds))

print("Testing classification report:")
print(classification_report(test_true, test_preds))
